In [1]:
import os
import psycopg2
import pandas as pd

In [ ]:
host = os.environ['DB_HOST']
port = os.environ['DB_PORT']
dbname = os.environ['DB_NAME']
user = os.environ['DB_USER']
password = os.environ['DB_PASSWORD']

In order to forecast churn for our _current_ customers, we first need to extract a metrics dataset consisting of just current customers, i.e. those that are active at present time.

The SQL script selects all accounts with at least 14 days' tenure. 
This constraint ensures that customers have been observed for a few weeks before
their metrics are used. If we don't do this, most new customers will have low metrics
due to the short observation period.

In [5]:
connection = psycopg2.connect(
    host=host,
    port=port,
    dbname=dbname,
    user=user,
    password=password
)

# read the SQL query template
query_path = 'create-revised-current-dataset' + '.sql'
with open(query_path, 'r') as file:
    sql_script = file.read()

# create a cursor to execute SQL queries
cursor = connection.cursor()

# prepare and execute the query
cursor.execute(sql_script)

# retrieve the results
results = cursor.fetchall()

# create a pandas DataFrame from the results
df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

# write to csv
csv_write_path = '../../output/socialnet7_dataset2_current' + '.csv'
df.to_csv(csv_write_path, index=False, mode='w')

# close the cursor and the connection
cursor.close()
connection.close()